In [1]:
pip install hvplot


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 4.1 MB/s eta 0:00:00


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important; }</style>"))

import glob
import pandas as pd
import numpy as np
import hvplot.pandas
import plotly.express as px
import plotly.graph_objects as go
pd.options.mode.chained_assignment = None

import datetime as dt
%autosave 30

Autosaving every 30 seconds


In [7]:
CPU_FREQ=2.60

rdtsc_df_dict = []
ttt_df_dict = []
for filename in glob.glob("./kse*.log"):
    print('processing {}'.format(filename))
    for line in open(filename):
        tokens = line.strip().split(' ')
        if len(tokens) != 4:
            continue

        try:
            time = tokens[0]
            tag = tokens[2]
            latency = float(tokens[3])
            latency_rdtsc = latency / CPU_FREQ
            time_datetime = pd.to_datetime(time, format='%H:%M:%S.%f')
        except:
            continue

        if ' RDTSC ' in line:
            if tokens[1] != 'RDTSC':
                continue

            rdtsc_df_dict.append({'timestamp':time, 'tag':tag, 'latency':latency_rdtsc})
        elif ' TTT ' in line:
            if tokens[1] != 'TTT':
                continue

            ttt_df_dict.append({'timestamp':time, 'tag':tag, 'latency':latency})

rdtsc_df = pd.DataFrame.from_dict(rdtsc_df_dict)
rdtsc_df = rdtsc_df.drop_duplicates().sort_values(by='timestamp')
rdtsc_df['timestamp'] = pd.to_datetime(rdtsc_df['timestamp'], format='%H:%M:%S.%f')

ttt_df = pd.DataFrame.from_dict(ttt_df_dict)
ttt_df = ttt_df.drop_duplicates().sort_values(by='timestamp')
ttt_df['timestamp'] = pd.to_datetime(ttt_df['timestamp'], format='%H:%M:%S.%f')

processing ./kse_matching_engine.log
processing ./kse_order_server.log
processing ./kse_market_data_publisher.log


In [8]:
for tag in rdtsc_df['tag'].unique():
    print(tag)

    fig = go.Figure()

    t_df = rdtsc_df[rdtsc_df['tag'] == tag].copy()
    t_df = t_df[t_df['latency'] > 0]

    q_hi = t_df['latency'].quantile(0.99)
    q_lo = t_df['latency'].quantile(0.01)
    t_df = t_df[(t_df['latency'] < q_hi) & (t_df['latency'] > q_lo)]

    mean = t_df['latency'].astype(float).mean()
    print('{} has {} observations mean {}'.format(tag, len(t_df), mean))

    rolling_window = max(1, int(len(t_df) / 100))

    use_micros = False
    if mean >= 1000:
        use_micros = True
        t_df['latency'] = t_df['latency'].astype(float) / 1000

    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency'], name=tag))
    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency'].rolling(rolling_window).mean(), name=tag + ' mean'))
#     fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency'].rolling(rolling_window).std(), name=tag + ' std'))

    fig.update_layout(title='performance ' + tag + ' ' + ('microseconds' if use_micros else 'nanoseconds'), height=750, width=1000, hovermode='x', legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))
    fig.show()

Exchange_odsSerialization
Exchange_odsSerialization has 1125 observations mean 124.87863247863247


Exchange_odsDeserialization
Exchange_odsDeserialization has 626 observations mean 31.796510199066105


Exchange_FIFOSequencer_addClientRequest
Exchange_FIFOSequencer_addClientRequest has 615 observations mean 54.80550343964977


Exchange_FIFOSequencer_sequenceAndPublish
Exchange_FIFOSequencer_sequenceAndPublish has 561 observations mean 36910.89537913067


Exchange_MEOrderBook_checkForMatch
Exchange_MEOrderBook_checkForMatch has 518 observations mean 58690.44772794772


...

Exchange_MatchingEngine_processClientRequest
Exchange_MatchingEngine_processClientRequest has 626 observations mean 138223.6163676579


Exchange_mdpubSerialization
Exchange_mdpubSerialization has 1013 observations mean 97.45614701192194


Exchange_MEOrderBook_removeOrder
Exchange_MEOrderBook_removeOrder has 250 observations mean 707.5507692307691


Exchange_MEOrderBook_cancel
Exchange_MEOrderBook_cancel has 106 observations mean 98471.46226415092


Exchange_MEOrderBook_match
Exchange_MEOrderBook_match has 249 observations mean 132402.5718257646


In [11]:
HOPS = [
    ['T1_OrderServer_TCP_read', 'T2_OrderServer_LFQueue_write'],
    ['T2_OrderServer_LFQueue_write', 'T3_MatchingEngine_LFQueue_read'],
    ['T3_MatchingEngine_LFQueue_read', 'T4_MatchingEngine_LFQueue_write'], ['T3_MatchingEngine_LFQueue_read', 'T4t_MatchingEngine_LFQueue_write'],
    ['T4_MatchingEngine_LFQueue_write', 'T5_MarketDataPublisher_LFQueue_read'], ['T4t_MatchingEngine_LFQueue_write', 'T5t_OrderServer_LFQueue_read'],
    ['T5_MarketDataPublisher_LFQueue_read', 'T6_MarketDataPublisher_UDP_write'], ['T5t_OrderServer_LFQueue_read', 'T6t_OrderServer_TCP_write'], ['T1_OrderServer_TCP_read', 'T6t_OrderServer_TCP_write'],
]

In [12]:
for tags in HOPS:
    tag_p, tag_n = tags
    print('{} => {}. {} => {}.'.format(tag_p, len(ttt_df[ttt_df['tag'] == tag_p]), tag_n, len(ttt_df[ttt_df['tag'] == tag_n])))

    fig = go.Figure()

    t_df = ttt_df[(ttt_df['tag'] == tag_n) | (ttt_df['tag'] == tag_p)]
    t_df['latency_diff'] = t_df['latency'].diff()
    t_df = t_df[t_df['latency_diff'] > 0]
    t_df = t_df[t_df.tag == tag_n]

    q_hi = t_df['latency_diff'].quantile(0.99)
    q_lo = t_df['latency_diff'].quantile(0.01)
    t_df = t_df[(t_df['latency_diff'] < q_hi) & (t_df['latency_diff'] > q_lo)]

    mean = t_df['latency_diff'].astype(float).mean()
    print('{} has {} observations mean {}'.format(tag_n, len(t_df), mean))

    rolling_window = max(1, int(len(t_df) / 100))

    unit = 'nanoseconds'
    if mean >= 1000000:
        unit = 'milliseconds'
        t_df['latency_diff'] = t_df['latency_diff'].astype(float) / 1000000
    elif mean >= 1000:
        unit = 'microseconds'
        t_df['latency_diff'] = t_df['latency_diff'].astype(float) / 1000

    tag_name = tag_p + ' -> ' + tag_n
    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency_diff'], name=tag_name))
    fig.add_trace(go.Scatter(x=t_df['timestamp'], y=t_df['latency_diff'].rolling(rolling_window).mean(), name=tag_name + ' mean'))

    fig.update_layout(title='performance ' + tag_name + ' ' + unit, height=750, width=1000, hovermode='x', legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))
    fig.show()


T1_OrderServer_TCP_read => 603. T2_OrderServer_LFQueue_write => 640.
T2_OrderServer_LFQueue_write has 626 observations mean 136242.30031948883


T2_OrderServer_LFQueue_write => 640. T3_MatchingEngine_LFQueue_read => 640.
T3_MatchingEngine_LFQueue_read has 555 observations mean 36551.26486486487


T3_MatchingEngine_LFQueue_read => 640. T4_MatchingEngine_LFQueue_write => 1040.
T4_MatchingEngine_LFQueue_write has 1018 observations mean 109166.39685658153


T3_MatchingEngine_LFQueue_read => 640. T4t_MatchingEngine_LFQueue_write => 1150.
T4t_MatchingEngine_LFQueue_write has 1125 observations mean 68403.88266666667


T4_MatchingEngine_LFQueue_write => 1040. T5_MarketDataPublisher_LFQueue_read => 1040.
T5_MarketDataPublisher_LFQueue_read has 1007 observations mean 118641.38232373387


T4t_MatchingEngine_LFQueue_write => 1150. T5t_OrderServer_LFQueue_read => 1154.
T5t_OrderServer_LFQueue_read has 1100 observations mean 92169.54181818182


T5_MarketDataPublisher_LFQueue_read => 1040. T6_MarketDataPublisher_UDP_write => 671.
T6_MarketDataPublisher_UDP_write has 656 observations mean 720080.7804878049


T5t_OrderServer_LFQueue_read => 1154. T6t_OrderServer_TCP_write => 693.
T6t_OrderServer_TCP_write has 678 observations mean 167715.87020648966


T1_OrderServer_TCP_read => 603. T6t_OrderServer_TCP_write => 693.
T6t_OrderServer_TCP_write has 677 observations mean 553188.3958641064


In [ ]:
import session_info
session_info.show()